In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Function for simulating Geometric Brownian Motion
def gbm(n_years=10, n_scenarios=1000, mu=0.07, sigma=0.15, steps_per_year=255, s_0=100.0, prices=True):
    dt = 1 / steps_per_year
    n_steps = int(n_years * steps_per_year) + 1
    rets_plus_1 = np.random.normal(loc=(1 + mu)**dt, scale=(sigma * np.sqrt(dt)), size=(n_steps, n_scenarios))
    rets_plus_1[0] = 1
    prices = s_0 * pd.DataFrame(rets_plus_1).cumprod() if prices else rets_plus_1 - 1
    return prices

# Function for applying the CPPI strategy
def cppi(gbm_sim, floor=80, multiplier=3, risk_free_rate=0.02, rebalance_per_year=12):
    n_steps, n_scenarios = gbm_sim.shape
    
    # Calculer le nombre d'étapes entre les rééquilibrages
    steps_per_rebalance = max(1, int(gbm_sim.index.size / (rebalance_per_year * (gbm_sim.index[-1] / 12))))
    
    account_value = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    cushion = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    risky_weight = pd.DataFrame(index=gbm_sim.index, columns=gbm_sim.columns)
    
    account_value.iloc[0] = gbm_sim.iloc[0]
    
    for step in range(1, n_steps):
        # Vérifier si c'est un moment de rééquilibrage
        if step % steps_per_rebalance == 0:
            cushion.iloc[step - 1] = np.maximum(account_value.iloc[step - 1] - floor, 0)
            risky_weight.iloc[step - 1] = multiplier * cushion.iloc[step - 1] / account_value.iloc[step - 1]
            risky_weight.iloc[step - 1] = np.clip(risky_weight.iloc[step - 1], 0, 1)
        else:
            risky_weight.iloc[step - 1] = risky_weight.iloc[step - 2] if step > 1 else 0
        
        safe_weight = 1 - risky_weight.iloc[step - 1]
        account_value.iloc[step] = (
            account_value.iloc[step - 1] * 
            (risky_weight.iloc[step - 1] * (gbm_sim.iloc[step] / gbm_sim.iloc[step - 1]) +
             safe_weight * (1 + risk_free_rate / rebalance_per_year))
        )
    
    return account_value

In [64]:
# Interactive widgets
n_years_widget = widgets.IntSlider(value=5, min=1, max=20, step=1, description='Years')
n_scenarios_widget = widgets.IntSlider(value=100, min=10, max=1000, step=10, description='Scenarios')
mu_widget = widgets.FloatSlider(value=0.07, min=0.0, max=0.2, step=0.01, description='Mu')
sigma_widget = widgets.FloatSlider(value=0.15, min=0.0, max=0.5, step=0.01, description='Sigma')

floor_widget = widgets.FloatSlider(value=90, min=50, max=100, step=1, description='Floor')
multiplier_widget = widgets.IntSlider(value=4, min=1, max=10, step=1, description='Multiplier')
rf_rate_widget = widgets.FloatSlider(value=0.04, min=0.0, max=0.1, step=0.01, description='Risk-free Rate')
rebalance_widget = widgets.IntSlider(value=10, min=1, max=255, step=1, description='Rebalance/Year')

# Button to run the simulation
run_button = widgets.Button(description="Run Simulation", button_style='success')

# Output widget to display results
output = widgets.Output()

# Function to run the simulation and plot results
def run_simulation(event):
    with output:
        output.clear_output()
        
        # Récupérer les valeurs des widgets
        n_years = n_years_widget.value
        n_scenarios = n_scenarios_widget.value
        mu = mu_widget.value
        sigma = sigma_widget.value
        steps_per_year = 255
        s_0 = 100
        
        floor = floor_widget.value
        multiplier = multiplier_widget.value
        rf_rate = rf_rate_widget.value
        rebalance_per_year = rebalance_widget.value
        
        # Exécuter la simulation GBM
        gbm_sim = gbm(
            n_years=n_years, 
            n_scenarios=n_scenarios, 
            mu=mu, 
            sigma=sigma, 
            steps_per_year=steps_per_year,
            s_0=s_0,
            prices=True
        )
        
        # Appliquer la stratégie CPPI
        portfolio_values = cppi(
            gbm_sim, 
            floor=floor, 
            multiplier=multiplier, 
            risk_free_rate=rf_rate, 
            rebalance_per_year=rebalance_per_year
        )
        
        # Calculer la moyenne du portefeuille CPPI
        cppi_mean = portfolio_values.mean(axis=1)
        
        # Tracer les résultats
        plt.figure(figsize=(14, 7))
        
        # Tracer chaque simulation en gris
        plt.plot(portfolio_values, color='grey', linewidth=0.5, alpha=0.25)
        
        # Tracer la ligne du floor
        plt.axhline(y=floor, linestyle='--', color='red', label='Floor')
        
        # Tracer la moyenne du portefeuille CPPI
        plt.plot(cppi_mean, color='blue', alpha=0.75, label='CPPI Portfolio (Mean)')
        
        # Personnaliser le graphique
        plt.yscale('log')
        plt.title("CPPI Strategy Simulation")
        plt.xlabel("Time Steps")
        plt.ylabel("Portfolio Value (log scale)")
        plt.grid(True)
        plt.legend()
        plt.show()

# Bind the button to the function
run_button.on_click(run_simulation)

# Display the widgets
display(
    n_years_widget, 
    n_scenarios_widget, 
    mu_widget, 
    sigma_widget, 
    floor_widget, 
    multiplier_widget, 
    rf_rate_widget, 
    rebalance_widget,
    run_button, output
)

IntSlider(value=5, description='Years', max=20, min=1)

IntSlider(value=100, description='Scenarios', max=1000, min=10, step=10)

FloatSlider(value=0.07, description='Mu', max=0.2, step=0.01)

FloatSlider(value=0.15, description='Sigma', max=0.5, step=0.01)

FloatSlider(value=90.0, description='Floor', min=50.0, step=1.0)

IntSlider(value=4, description='Multiplier', max=10, min=1)

FloatSlider(value=0.04, description='Risk-free Rate', max=0.1, step=0.01)

IntSlider(value=10, description='Rebalance/Year', max=255, min=1)

Button(button_style='success', description='Run Simulation', style=ButtonStyle())

Output()

In [126]:
data_df = cppi(
    gbm(
        n_years=3, 
        n_scenarios=100, 
        mu=0.05, 
        sigma=0.15,
        steps_per_year=25,
        s_0=100, 
        prices=True
    ),
    floor=80, 
    multiplier=3, 
    risk_free_rate=0.03, 
    rebalance_per_year=3
)

In [127]:
def avg_return(history: pd.Series, timeperiod: int = 255) -> float:
    returns = history.pct_change().dropna()
    return (1 + returns.sum()) ** (timeperiod/returns.count()) - 1

def avg_volatility(history: pd.Series, timeperiod: int = 255) -> float:
    returns = history.pct_change().dropna()
    return np.std(returns, axis=1) * np.sqrt(timeperiod)

def max_drawdown(history: pd.Series) -> float:
    index = 100 * (1 + history.pct_change()).cumprod()
    peaks = index.cummax()
    drawdowns = (index - peaks) / peaks
    return drawdowns.min()

result_df = pd.DataFrame()
result_df['Max drawdown'] = max_drawdown(data_df).describe().round(4)
result_df['Average return'] = avg_return(data_df, timeperiod=25).describe().round(4)
result_df['Average volatility'] = avg_volatility(data_df.T, timeperiod=25).describe().round(4)
result_df

,Max drawdown,Average return,Average volatility
count,100.0000,100.0000,99.0000
mean,-0.1501,0.0819,0.5208
std,0.0436,0.0488,0.2420
min,-0.3055,-0.0224,0.2338
25%,-0.1842,0.0471,0.3787
50%,-0.1442,0.0753,0.4789
75%,-0.1122,0.1071,0.5949
max,-0.0699,0.1946,1.9714


In [124]:
avg_return(data_df)

0     1.716199
1     5.894764
2     1.239813
3     5.148683
4     3.784220
        ...   
95    1.410746
96    4.367593
97   -0.233936
98    2.911297
99   -0.216272
Length: 100, dtype: float64

In [107]:
data_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
1,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0,...,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0,101.0
2,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01,...,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01,102.01
3,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,...,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301,103.0301
4,99.473973,104.997345,105.189422,102.833419,111.843102,106.083512,108.281343,109.316366,102.203661,111.462386,...,105.598914,111.766613,103.751594,99.971278,109.498194,101.134585,103.284767,106.894785,109.451018,101.006729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,110.251387,104.051409,94.448338,147.270839,101.442637,106.219179,232.789925,107.538693,116.298868,308.770008,...,133.876433,107.435246,229.495326,115.672122,103.04411,165.79237,97.471342,122.767583,99.300241,211.300183
72,116.547827,112.119582,94.315257,153.148702,99.019639,101.360757,205.369034,103.952256,102.697752,329.15084,...,124.342107,101.833169,218.585182,107.7894,109.595098,157.151842,97.694296,110.105298,101.812584,187.53314
73,117.558312,114.956992,89.9656,149.950014,101.859105,99.905159,216.349829,102.164187,103.154965,336.951069,...,118.184142,102.26616,240.212693,104.927676,109.431614,168.248323,104.673467,114.507527,104.815717,170.154222
74,113.617903,118.72399,91.352538,156.853096,110.207098,90.407955,210.508752,101.442527,100.462681,351.358683,...,116.658481,105.866319,224.115097,101.674606,111.20208,150.008,112.546724,99.901661,107.55439,183.664127
